In [6]:
#  A simple function

import json
def get_current_weather(location: str,unit: str):
    if location=="Madrid":
        return json.dumps({"temperature":25,"unit":unit})
    else:
        return json.dumps({"temperature":58,"unit":unit})
get_current_weather("Madrid","celsius")


'{"temperature": 25, "unit": "celsius"}'

In [29]:
import os
import re
from groq import Groq
from dotenv import load_dotenv

load_dotenv()

MODEL = "llama3-70b-8192"
client  = Groq()

TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

In [30]:
tool_chat_history = [
    {
        "role":"system",
        "content": TOOL_SYSTEM_PROMPT
    }
]

agent_chat_history = []

user_msg = {
    "role":"user",
    "content":"What's the current temperature in Madrid, in Celsius?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)


output = client.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

In [31]:
output

'<tool_call>\n{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}}\n</tool_call>'

In [39]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern,'',tool_call_str)
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [41]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Madrid', 'unit': 'celsius'}}

In [43]:
result = get_current_weather(**parsed_output["arguments"])
print(result)

{"temperature": 25, "unit": "celsius"}


In [44]:
agent_chat_history.append({
    "role":"user",
    "content":f"Observation: {result}"
})

In [45]:
client.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current temperature in Madrid is 25°C.'


**The tool decorator**
We are going to use the '''tool''' decorator to transform any Python function into a tool. You can see the implementation here. To test it out, let's make a more complex tool than before. For example, a tool that interacts with Hacker News, getting the current top stories.

Reminder: To automatically generate the function signature for the tool, we need a way to infer the arguments types. For this reason, we need to create the typing annotations.

In [49]:
import json
import requests
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

In [52]:
def fetch_top_hacker_news_stories(top_n: int):
    top_stories_url='https://hacker-news.firebaseio.com/v0/topstories.json'

    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()
        top_story_ids = response.json()[:top_n]
        top_stories=[]

        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()
            story_data = story_response.json()
            top_stories.append({
                'title': story_data.get('title','No title'),
                'url': story_data.get('url','No URL available')
            })
        return json.dumps(top_stories)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []
            

In [53]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'My 16-month theanine self-experiment',
  'url': 'https://dynomight.net/theanine/'},
 {'title': 'Show HN: I built an app to get daily wisdom from Mr. Worldwide',
  'url': 'https://daale.club/'},
 {'title': 'Posthog/.cursorrules',
  'url': 'https://github.com/PostHog/posthog/blob/master/.cursorrules'},
 {'title': 'Presenterm: Markdown Slideshows in the Terminal',
  'url': 'https://github.com/mfontanini/presenterm'},
 {'title': 'AI tools are spotting errors in research papers',
  'url': 'https://www.nature.com/articles/d41586-025-00648-5'}]

In [58]:
hn_tool = tool(fetch_top_hacker_news_stories)
wt_tool = tool(get_current_weather)

In [56]:
hn_tool.name

'fetch_top_hacker_news_stories'

In [57]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_stories',
 'description': None,
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

In [62]:
wt_tool.fn_signature

'{"name": "get_current_weather", "description": null, "parameters": {"properties": {"location": {"type": "str"}, "unit": {"type": "str"}}}}'

**The ToolAgent**
To create the agent, we just need to pass a list of tools (in this case, just one).

In [65]:
tool_agent = ToolAgent(model="llama3-70b-8192",tools=[hn_tool,wt_tool])

In [71]:
output = tool_agent.run(user_msg="What is the capital of India?")
print(output)

The capital of India is New Delhi.


In [72]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "My 16-month theanine self-experiment", "url": "https://dynomight.net/theanine/"}, {"title": "Posthog/.cursorrules", "url": "https://github.com/PostHog/posthog/blob/master/.cursorrules"}, {"title": "Show HN: I built an app to get daily wisdom from Mr. Worldwide", "url": "https://daale.club/"}, {"title": "AI tools are spotting errors in research papers", "url": "https://www.nature.com/articles/d41586-025-00648-5"}, {"title": "Presenterm: Markdown Slideshows in the Terminal", "url": "https://github.com/mfontanini/presenterm"}]


In [68]:
print(output)

Here are the top 5 Hacker News stories right now:

1. **My 16-month theanine self-experiment** - https://dynomight.net/theanine/
2. **Show HN: I built an app to get daily wisdom from Mr. Worldwide** - https://daale.club/
3. **Posthog/.cursorrules** - https://github.com/PostHog/posthog/blob/master/.cursorrules
4. **Presenterm: Markdown Slideshows in the Terminal** - https://github.com/mfontanini/presenterm
5. **AI tools are spotting errors in research papers** - https://www.nature.com/articles/d41586-025-00648-5


In [70]:
output = tool_agent.run(user_msg="Whats the weather in Madrid in Celsius?")


Using Tool: get_current_weather

Tool call dict: 
{'name': 'get_current_weather', 'arguments': {'location': 'Madrid', 'unit': 'Celsius'}, 'id': 1}

Tool result: 
{"temperature": 25, "unit": "Celsius"}
